# Imports

In [1]:
# !pip install -q git+https://github.com/tensorflow/docs

In [2]:
# !pip install imutils

In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [4]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [5]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 300000

# Define path

In [6]:
#trainfilepath = ("/content/gdrive/MyDrive/DissertationVideosSorted/train.csv")
#testfilepath = ("/content/gdrive/MyDrive/DissertationVideosSorted/test.csv")

In [7]:
trainfilepath = ("./vdataset/train.csv")
testfilepath = ("./vdataset/test.csv")

In [8]:
train_df = pd.read_csv(trainfilepath)
test_df = pd.read_csv(testfilepath)

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 58
Total videos for testing: 2


,video_name,tag
28,/Users/whitepheonix/Downloads/vdataset/notdrow...,not drowning
20,/Users/whitepheonix/Downloads/vdataset/notdrow...,not drowning
41,/Users/whitepheonix/Downloads/vdataset/drownin...,drowning
14,/Users/whitepheonix/Downloads/vdataset/notdrow...,not drowning
7,/Users/whitepheonix/Downloads/vdataset/notdrow...,not drowning
42,/Users/whitepheonix/Downloads/vdataset/drownin...,drowning
5,/Users/whitepheonix/Downloads/vdataset/notdrow...,not drowning
46,/Users/whitepheonix/Downloads/vdataset/drownin...,drowning
16,/Users/whitepheonix/Downloads/vdataset/notdrow...,not drowning
55,/Users/whitepheonix/Downloads/vdataset/drownin...,drowning


# Data Processing Pipeline

In [9]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

# Feature Extractor

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

input_shape=(IMG_SIZE, IMG_SIZE, 3)

def build_feature_extractor():
    feature_extractor = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='relu', input_shape=input_shape, padding='same'), #C1
    keras.layers.AveragePooling2D(), #S2
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='relu', padding='valid'), #C3
    keras.layers.AveragePooling2D(), #S4
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='relu', padding='valid'), #C5
    keras.layers.Flatten(), #Flatten    
])
    feature_extractor.summary()

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    
    outputs = feature_extractor(inputs)

    return keras.Model(inputs, outputs, name="feature_extractor")

feature_extractor = build_feature_extractor()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 6)       456       
                                                                 
 average_pooling2d (AverageP  (None, 112, 112, 6)      0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 16)      2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 54, 54, 16)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 120)       48120     
                                                                 
 flatten (Flatten)           (None, 300000)            0

2022-08-16 00:07:18.691826: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['drowning', 'not drowning']


In [14]:
trainrootdir = ("./vdataset")
testrootdir=("./vdataset")


In [15]:
from tqdm import tqdm
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in tqdm(enumerate(video_paths)):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, trainrootdir)
test_data, test_labels = prepare_all_videos(test_df, testrootdir)

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

58it [01:29,  1.55s/it]
2it [00:03,  1.83s/it]

Frame features in train set: (58, 20, 300000)
Frame masks in train set: (58, 20)


# LSTM Model and Evaluation

In [16]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, cohen_kappa_score, classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")


    x = keras.layers.LSTM(8, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.LSTM(8)(x)
    #x = keras.layers.Dropout(0.3)(x)

    x = keras.layers.Dense(30, activation="relu")(x)
    x = keras.layers.Dense(20, activation="relu")(x)
    x = keras.layers.Dense(10, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=["accuracy"],
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1#, monitor="accuracy"
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        #validation_split=0.2,
        validation_data=([test_data[0], test_data[1]], test_labels), 
        epochs=150,
        callbacks=[checkpoint]
    )

   #Plot Graphs
    plt.figure(0)
    plt.plot(history.history['accuracy'], label='Training accuracy')
    plt.plot(history.history['val_accuracy'], label='Val accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.figure(1)
    plt.plot(history.history['loss'], label='Training loss')
    plt.plot(history.history['val_loss'], label='Val loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    seq_model.summary()

    
    #Experiments Summary
    seq_model.load_weights(filepath)
    model_results = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(model_results[1] * 100, 2)}%")
    print("Test loss:" + str(model_results[0]))
    


    test_pred = seq_model.predict([test_data[0], test_data[1]])
    test_pred = list(np.argmax(test_pred,axis = 1))

   
    #Cohen's Kappa and F1 Score
    ConfusionMatrixDisplay.from_predictions(test_labels, test_pred)

    print("Cohen's Kappa:"+str (cohen_kappa_score(test_labels, test_pred)))
    print(classification_report(test_labels, test_pred))

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/150
2/2 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.5000 
Epoch 00001: val_loss improved from inf to 0.69624, saving model to video_classifier
2/2 [==============================] - 17s 5s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6962 - val_accuracy: 0.5000
Epoch 2/150
2/2 [==============================] - ETA: 0s - loss: 0.6964 - accuracy: 0.5000
Epoch 00002: val_loss improved from 0.69624 to 0.67464, saving model to video_classifier
2/2 [==============================] - 6s 3s/step - loss: 0.6964 - accuracy: 0.5000 - val_loss: 0.6746 - val_accuracy: 0.5000
Epoch 3/150
2/2 [==============================] - ETA: 0s - loss: 0.6987 - accuracy: 0.5000
Epoch 00003: val_loss did not improve from 0.67464
2/2 [==============================] - 3s 1s/step - loss: 0.6987 - accuracy: 0.5000 - val_loss: 0.6958 - val_accuracy: 0.5000
Epoch 4/150
2/2 [==============================] - ETA: 0s - loss: 0.6955 - accuracy: 0.5000
Epoch 00004: val_loss d

Epoch 31/150
2/2 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5000
Epoch 00031: val_loss did not improve from 0.67464
2/2 [==============================] - 3s 1s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 32/150
2/2 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5000
Epoch 00032: val_loss did not improve from 0.67464
2/2 [==============================] - 3s 1s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 33/150
2/2 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5000
Epoch 00033: val_loss did not improve from 0.67464
2/2 [==============================] - 3s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 34/150
2/2 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5000
Epoch 00034: val_loss did not improve from 0.67464
2/2 [==============================] - 3s 